I am tasked with working for an investor who specializes in purchasing undervalued assets. The investor wants to perform detailed data analysis to identify opportunities for growth and profitability in a potential purchase—TellCo, a mobile service provider in the Republic of Pefkakia. Your goal is to analyze customer data, provide insights, and make recommendations on whether TellCo is worth buying or selling. The analysis will be presented through a web-based dashboard and a written report.

          Task 3: Experience Analytics
Objective: Evaluate customer experience based on network parameters and device characteristics.

Aggregate average TCP retransmission, RTT, handset type, and throughput per customer.

List top, bottom, and most frequent TCP, RTT, and throughput values.

Report distributions and averages of throughput and TCP retransmission per handset type.

Perform k-means clustering to segment users into experience groups and describe each cluster.

